## What is a FrontDoor Causal Bootstrap Algorithm?

- It is an algorithm which resamples the dataset to create a new dataset using the frontdoor criterion
- The idea here is that a problem won't have only a single confounder but multiple instead
- which therefore may not be observable
- which is why we look at variables which sit on a causal path X -> Z -> Y known as **mediators**
- the front door criterion applies where you have a known mediator Z which is observed and there are no  confounders associating X -> Z and Z -> Y

**Example**
- Imagine you want to assess whether studying more hours (the “cause” X) leads to improved exam performance (the “effect” Y). 
- However there are many unobserved confounders such as "prior knowledge" and "innate ability" which
may not be observed so backdoor causal bootstrapping isn't possible
- However lets say we have a known variable "amount of practice problems done"
- we know that people who study more do more practice problems and people who do more practice problems get better grades
- and if we know there are no variables which influence both studying more and doing more practice problems
- and we also know there are no variables which influence both doing more practice problems and doing better in exams
- then we can apply front door causal bootstrapping to really understand the relation between the input and output variables

In [13]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

def _ensure_2d(arr):
    arr = np.asarray(arr)
    if arr.ndim == 1:
        arr = arr.reshape(-1, 1)
    return arr

def gaussian_kernel_matrix(A, B=None, bandwidth=1.0):
    """
    Gaussian/RBF kernel matrix: K_ij = exp(-0.5 * ||a_i - b_j||^2 / h^2)
    A: (n,d), B: (m,d)
    """
    A = _ensure_2d(A)
    if B is None:
        B = A
    else:
        B = _ensure_2d(B)
    dists = cdist(A, B, metric="euclidean")
    return np.exp(-0.5 * (dists / float(bandwidth)) ** 2)

def kernel_y_vector(y_data, y_star, *, discrete=True, bandwidth_y=1.0):
    """
    K[y_i - y*] used in the paper.
    - discrete=True -> Kronecker delta (1 if equal else 0)
    - discrete=False -> Gaussian kernel on (y_i - y*)
    """
    y_data = np.asarray(y_data)
    if discrete:
        return (y_data == y_star).astype(float)
    return np.exp(-0.5 * ((y_data - y_star) / float(bandwidth_y)) ** 2)

def phat_y_given_S(y_data, S_data, y_star, *, discrete_y=True, bandwidth_S=1.0, bandwidth_y=1.0, eps=1e-12):
    """
    Nonparametric estimate of p_hat(y* | S_i) for each i, using kernel regression:
      p_hat(y*|S_i) = sum_j K_S(S_i, S_j) * K_Y(y_j, y*) / sum_j K_S(S_i, S_j)
    Returns: (N,) vector over i.
    """
    K_S = gaussian_kernel_matrix(S_data, bandwidth=bandwidth_S)  # (N,N)
    K_Y = kernel_y_vector(y_data, y_star, discrete=discrete_y, bandwidth_y=bandwidth_y)  # (N,)
    numer = K_S @ K_Y
    denom = K_S.sum(axis=1)
    return numer / np.maximum(denom, eps)

def phat_z_given_y(z_data, y_data, *, bandwidth_z=1.0, eps=1e-12):
    """
    For front-door (Algorithm 2): estimate p_hat(z_i | y=v) for all i and each discrete y=v.
    KDE on z within each y-group:
      p_hat(z_i | y=v) ∝ (1/N_v) * sum_{j:y_j=v} K_z(z_i, z_j)
    Returns: dict mapping y_value -> (N,) vector, entry i is p_hat(z_i | y_value)
    """
    z = _ensure_2d(z_data)
    y = np.asarray(y_data)
    y_vals = np.unique(y)

    Kzz = gaussian_kernel_matrix(z, bandwidth=bandwidth_z)  # (N,N)
    phat = {}
    for v in y_vals:
        mask = (y == v)
        Nv = int(mask.sum())
        if Nv == 0:
            phat[v] = np.full(len(y), eps)
            continue
        numer = Kzz[:, mask].sum(axis=1) / float(Nv)
        phat[v] = np.maximum(numer, eps)
    return phat


## Choose columns

- `y_col`: intervention / prediction target **Y** (assumed *discrete* here, matching Algorithm 2)
- `z_cols`: mediator(s) **Z**
- `x_cols`: features **X** used to train your model (typically exclude Z so you learn the effect of Y on X, not the mediation variable itself)

Kernel settings:
- `bandwidth_z` controls KDE smoothness on mediator space.


In [ ]:
# --- USER INPUTS ---
df = pd.read_csv("heart_disease_preprocessed.csv")  # change if needed

#looking at example mediators
y_col = "heartdiseasepresence"
z_cols = ["ca"]  # mediator(s) Z

# Default X: everything except Y and Z
x_cols = [c for c in df.columns if c not in [y_col]]

bandwidth_z = 1.0
random_seed = 0

print("y_col:", y_col)
print("z_cols:", z_cols)
print("x_cols:", x_cols)
print("N:", len(df))

y_col: heartdiseasepresence
z_cols: ['ca']
x_cols: ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'ca', 'sex_Female', 'sex_Male', 'cp_Asymptomatic', 'cp_AtypicalAngina', 'cp_NonAnginalPain', 'cp_TypicalAngina', 'fbs_<=120', 'fbs_>120', 'restecg_LVHypertrophy', 'restecg_NormalECG', 'restecg_STTAbnormality', 'exang_NoExAngina', 'exang_YesExAngina', 'slope_Downsloping', 'slope_Flat', 'slope_Upsloping', 'thal_FixedDefect', 'thal_Normal', 'thal_ReversibleDefect']
N: 272


In [15]:
def causal_bootstrap_frontdoor(df, *, x_cols, y_col, z_cols,
                             bandwidth_z=1.0, random_seed=0, eps=1e-12):
    """Front-door causal bootstrap (Algorithm 2), discrete Y.

    Returns dataframe with columns x_cols + [y_col], same number of rows as df.
    """
    N = len(df)
    y = df[y_col].to_numpy()
    z = df[z_cols].to_numpy(dtype=float)

    rng = np.random.default_rng(random_seed)
    y_vals = np.unique(y)

    # Precompute p_hat(z_i | y=v) for all v
    phat_z = phat_z_given_y(z, y, bandwidth_z=bandwidth_z, eps=eps)

    out_rows = []
    for y_star in y_vals:
        n_star = int((y == y_star).sum())
        if n_star == 0:
            continue

        # weights for each i: p_hat(z_i|y_star) / (N * p_hat(z_i|y_i))
        denom = np.array([phat_z[yi][i] for i, yi in enumerate(y)], dtype=float)
        numer = phat_z[y_star]
        w = numer / (float(N) * np.maximum(denom, eps))

        w_sum = w.sum()
        if w_sum <= 0:
            continue
        p = w / w_sum
        idx = rng.choice(np.arange(N), size=n_star, replace=True, p=p)

        block = df.iloc[idx][x_cols].copy()
        block[y_col] = y_star
        out_rows.append(block)

    df_star = pd.concat(out_rows, ignore_index=True)
    if len(df_star) != N:
        df_star = df_star.sample(n=N, replace=True, random_state=random_seed).reset_index(drop=True)
    return df_star

In [16]:
df_frontdoor = causal_bootstrap_frontdoor(
    df,
    x_cols=x_cols,
    y_col=y_col,
    z_cols=z_cols,
    bandwidth_z=bandwidth_z,
    random_seed=random_seed,
)

df_frontdoor.shape

(272, 26)

In [17]:
df_frontdoor

,age,trestbps,chol,thalach,oldpeak,ca,sex_Female,sex_Male,cp_Asymptomatic,cp_AtypicalAngina,...,restecg_STTAbnormality,exang_NoExAngina,exang_YesExAngina,slope_Downsloping,slope_Flat,slope_Upsloping,thal_FixedDefect,thal_Normal,thal_ReversibleDefect,heartdiseasepresence
0,1.397584,-1.806596,1.048884,-1.029668,0.120886,1.437899,0,1,1,0,...,0,0,1,0,1,0,0,1,0,0
1,0.392506,1.026953,0.460978,-1.629205,0.017112,-0.740979,0,1,1,0,...,0,0,1,0,0,1,0,0,1,0
2,0.169155,-0.106466,0.177162,-0.301658,-0.208954,0.348460,0,1,0,0,...,0,0,1,0,1,0,1,0,0,0
3,0.169155,-0.673176,-0.228291,1.240010,0.017112,-0.740979,0,1,0,1,...,0,1,0,0,0,1,0,1,0,0
4,1.397584,-0.673176,-0.208018,-3.342170,0.219335,-0.740979,0,1,1,0,...,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,0.727532,1.026953,-0.086382,-0.515779,0.219335,-0.740979,0,1,0,0,...,0,0,1,0,1,0,0,1,0,1
268,-0.054195,-1.239886,-0.167473,-0.986844,1.451274,0.348460,0,1,1,0,...,0,0,1,0,1,0,0,0,1,1
269,0.950883,0.743598,-0.289108,0.040935,1.180495,-0.740979,0,1,0,0,...,0,1,0,1,0,0,1,0,0,1
270,-2.287702,-0.786518,-0.755378,1.839547,-0.092595,-0.740979,1,0,0,1,...,0,1,0,0,0,1,0,1,0,1


In [18]:
out_path = "heart_disease_preprocessed_frontdoor.csv"
df_frontdoor.to_csv(out_path, index=False)